## Tut3 - RNN recognizing MNIST numbers

Based on: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb

A Reccurent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

For an alternative similar implementation, see https://github.com/nlintz/TensorFlow-Tutorials/blob/master/07_lstm.py

In [1]:
import os
import numpy as np
import tensorflow as tf
#from tensorflow.python.ops import rnn, rnn_cell

HOME = os.environ['HOME']
dpath = HOME + "/data/tf_tut/mnist/MNIST_data"

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(dpath, one_hot=True)

Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/t10k-labels-idx1-ubyte.gz


To classify images using a reccurent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.

**==> 28 seqs of 28 steps for every sample**

In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

### The Model

In [11]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
#    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell = tf.nn.rnn_cell.GRUCell(n_hidden)

    # Get lstm cell output
    outputs, states = tf.nn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

### Training and Evaluating the Net

#### Result with GRU Cells

In [12]:
%%time
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + 
                  "{:.6f}".format(loss) + ", Training Accuracy= " + 
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.948195, Training Accuracy= 0.28125
Iter 2560, Minibatch Loss= 1.665990, Training Accuracy= 0.44531
Iter 3840, Minibatch Loss= 1.316161, Training Accuracy= 0.59375
Iter 5120, Minibatch Loss= 1.130145, Training Accuracy= 0.60156
Iter 6400, Minibatch Loss= 0.917219, Training Accuracy= 0.69531
Iter 7680, Minibatch Loss= 0.925471, Training Accuracy= 0.70312
Iter 8960, Minibatch Loss= 0.764358, Training Accuracy= 0.73438
Iter 10240, Minibatch Loss= 0.627738, Training Accuracy= 0.78125
Iter 11520, Minibatch Loss= 0.522808, Training Accuracy= 0.82812
Iter 12800, Minibatch Loss= 0.587044, Training Accuracy= 0.79688
Iter 14080, Minibatch Loss= 0.457473, Training Accuracy= 0.85938
Iter 15360, Minibatch Loss= 0.504247, Training Accuracy= 0.83594
Iter 16640, Minibatch Loss= 0.381519, Training Accuracy= 0.90625
Iter 17920, Minibatch Loss= 0.278599, Training Accuracy= 0.92969
Iter 19200, Minibatch Loss= 0.335172, Training Accuracy= 0.89062
Iter 20480, Minibatch Loss= 0.37

#### Result with LSTM Cells

In [8]:
%%time
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + 
                  "{:.6f}".format(loss) + ", Training Accuracy= " + 
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.897562, Training Accuracy= 0.33594
Iter 2560, Minibatch Loss= 1.640621, Training Accuracy= 0.46094
Iter 3840, Minibatch Loss= 1.279966, Training Accuracy= 0.57031
Iter 5120, Minibatch Loss= 1.117690, Training Accuracy= 0.63281
Iter 6400, Minibatch Loss= 0.902289, Training Accuracy= 0.70312
Iter 7680, Minibatch Loss= 0.635122, Training Accuracy= 0.78906
Iter 8960, Minibatch Loss= 0.671882, Training Accuracy= 0.75781
Iter 10240, Minibatch Loss= 0.617051, Training Accuracy= 0.82031
Iter 11520, Minibatch Loss= 0.418730, Training Accuracy= 0.88281
Iter 12800, Minibatch Loss= 0.376057, Training Accuracy= 0.88281
Iter 14080, Minibatch Loss= 0.315794, Training Accuracy= 0.87500
Iter 15360, Minibatch Loss= 0.400999, Training Accuracy= 0.85156
Iter 16640, Minibatch Loss= 0.436320, Training Accuracy= 0.87500
Iter 17920, Minibatch Loss= 0.343609, Training Accuracy= 0.90625
Iter 19200, Minibatch Loss= 0.289465, Training Accuracy= 0.89844
Iter 20480, Minibatch Loss= 0.28